In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import Ipynb_importer
import data_util
import evaluation
import data_util

importing Jupyter notebook from data_util.ipynb
importing Jupyter notebook from evaluation.ipynb


In [2]:
#θ
alpha=0
attr_size=18
attr_present_dim=5
user_emb_dim=18 
hidden_dim=100
batch_size = 1024

In [3]:
#parameters of generative model G
G_attri_matrix = tf.get_variable('G_attri_matrix', [2*attr_size, attr_present_dim],initializer=tf.contrib.layers.xavier_initializer())
G_W1 = tf.get_variable('G_w1', [attr_size*attr_present_dim , hidden_dim],initializer=tf.contrib.layers.xavier_initializer())
G_b1 = tf.get_variable('G_b1', [1, hidden_dim],initializer=tf.contrib.layers.xavier_initializer())
G_W2 = tf.get_variable('G_w2', [hidden_dim, hidden_dim],initializer=tf.contrib.layers.xavier_initializer())
G_b2 = tf.get_variable('G_b2', [1, hidden_dim],initializer=tf.contrib.layers.xavier_initializer())
G_W3 = tf.get_variable('G_w3', [hidden_dim, user_emb_dim],initializer=tf.contrib.layers.xavier_initializer())
G_b3 = tf.get_variable('G_b3', [1, user_emb_dim],initializer=tf.contrib.layers.xavier_initializer())

G_params = [G_attri_matrix, G_W1, G_b1, G_W2, G_b2, G_W3, G_b3]

#parameters of discriminator model D
D_attri_matrix = tf.get_variable('D_attri_matrix', [2*attr_size, attr_present_dim],initializer=tf.contrib.layers.xavier_initializer())
D_W1 = tf.get_variable('D_w1', [attr_size*attr_present_dim  + user_emb_dim , hidden_dim],initializer=tf.contrib.layers.xavier_initializer())
D_b1 = tf.get_variable('D_b1', [1, hidden_dim],initializer=tf.contrib.layers.xavier_initializer())
D_W2 = tf.get_variable('D_w2', [hidden_dim, hidden_dim],initializer=tf.contrib.layers.xavier_initializer())
D_b2 = tf.get_variable('D_b2', [1, hidden_dim],initializer=tf.contrib.layers.xavier_initializer())
D_W3 = tf.get_variable('D_w3', [hidden_dim, user_emb_dim],initializer=tf.contrib.layers.xavier_initializer())
D_b3 = tf.get_variable('D_b3', [1, user_emb_dim],initializer=tf.contrib.layers.xavier_initializer())

D_params = [D_attri_matrix,D_W1, D_b1, D_W2, D_b2,D_W3,D_b3]



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
#generative model G
def model_G(attr):
    attr_present = tf.nn.embedding_lookup(G_attri_matrix, attr)
    attr_feat = tf.reshape(attr_present,shape=[-1,attr_size*attr_present_dim])
    
    layer1_output = tf.nn.tanh(tf.matmul( attr_feat, G_W1) + G_b1)   
    layer2_output = tf.nn.tanh(tf.matmul(layer1_output, G_W2) + G_b2)
    gen_user_emb = tf.nn.tanh(tf.matmul(layer2_output, G_W3) + G_b3)
    
    return gen_user_emb
    
#discriminator model D
def model_D(attr,user_emb):
    attr_present = tf.nn.embedding_lookup(G_attri_matrix, attr)
    attr_feat = tf.reshape(attr_present,shape=[-1,attr_size*attr_present_dim])
    emb = tf.concat([attr_feat, user_emb], 1)
    
    layer1_output = tf.nn.tanh(tf.matmul(emb, D_W1) + D_b1)
    layer2_output = tf.nn.tanh(tf.matmul(layer1_output, D_W2) + D_b2)
    D_logit = tf.matmul(layer2_output, D_W3) + D_b3
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob, D_logit

In [5]:
attr  = tf.placeholder(shape=[None,attr_size],dtype = tf.int32)
real_user_emb = tf.placeholder(shape = [None, user_emb_dim], dtype = tf.float32)

neg_attr  = tf.placeholder(shape=[None,attr_size],dtype = tf.int32)
neg_user_emb = tf.placeholder(shape = [None, user_emb_dim], dtype = tf.float32)

In [6]:
#loss
gen_user_emb = model_G(attr)
D_real, D_logit_real = model_D(attr, real_user_emb)
D_gen, D_logit_gen = model_D(attr, gen_user_emb)
D_neg, D_logit_neg = model_D(neg_attr, neg_user_emb)

D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_gen = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_gen, labels=tf.zeros_like(D_logit_gen)))
D_loss_neg = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_neg, labels=tf.zeros_like(D_logit_neg))) 

D_regular = alpha * (tf.nn.l2_loss(D_attri_matrix)  + tf.nn.l2_loss(D_W1) + tf.nn.l2_loss(D_b1) + tf.nn.l2_loss(D_W2) + tf.nn.l2_loss(D_b2) + tf.nn.l2_loss(D_W3) + tf.nn.l2_loss(D_b3)) 
G_regular = alpha * (tf.nn.l2_loss(G_attri_matrix)  + tf.nn.l2_loss(G_W1) + tf.nn.l2_loss(G_b1) + tf.nn.l2_loss(G_W2) + tf.nn.l2_loss(G_b2) + tf.nn.l2_loss(G_W2) + tf.nn.l2_loss(G_b2)+ tf.nn.l2_loss(G_W3)+ tf.nn.l2_loss(G_b3) )

D_loss = (1-alpha)*(D_loss_real + D_loss_gen  + D_loss_neg)+ D_regular
G_loss = (1-alpha)*(tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_gen, labels=tf.ones_like(D_logit_gen)))) + G_regular    

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
'''optimizer'''
D_solver = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(D_loss, var_list=D_params)
G_solver = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(G_loss, var_list=G_params)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())


test_item_batch,  test_attribute_vec= data_util.get_test_data()

test_G_user = sess.run(gen_user_emb, feed_dict={attr:test_attribute_vec})

p_at_10,p_at_20,M_at_10,M_at_20,G_at_10,G_at_20 = data_util.test(test_item_batch, test_G_user)
print('at start p_at_10 is', p_at_10, 'p_at_20', p_at_20,'M_at_10',M_at_10,'M_at_20',M_at_20,'G_at_10',G_at_10,'G_at_20',G_at_20)


max_p_at_10 = 0
max_p_at_20 = 0
max_M_at_10 = 0
max_M_at_20 = 0 
max_G_at_10 = 0
max_G_at_20 = 0

train_data = np.array(pd.read_csv('Data/train_data.csv')) 

np.random.shuffle(train_data)
user_emb_matrix = np.array(pd.read_csv('generate_data/user_attribute_normal.csv',header=None))

p_10_to_save = []
p_20_to_save = []
M_10_to_save = []
M_20_to_save = []
G_10_to_save = []
G_20_to_save = []
 
'''train'''
for it in range(800):
    D_range = 1
    G_range = 1    
    
    data_util.shuffle_train()
    data_util.shuffle_neg()
    for D_it in range(D_range):
        index = 0

        while index < 253236:
            if index + batch_size <= 253236:
                train_user_batch, train_item_batch, train_attr_batch,train_user_emb_batch = data_util.get_train_data(index, index + batch_size)
                counter_user_batch, counter_item_batch, counter_attr_batch, counter_user_emb_batch = data_util.get_neg_data(index, index + batch_size)
            index = index + batch_size
               
            _, D_loss_now,fake_us= sess.run([D_solver, D_loss,gen_user_emb], 
                                      feed_dict={attr : train_attr_batch,
                                                 real_user_emb : train_user_emb_batch,
                                                 neg_attr  :  counter_attr_batch,
                                                 neg_user_emb : counter_user_emb_batch                                                 
                                               })  
        print( D_loss_now)

    for G_it in range(G_range):
        index = 0
        while index < 253236:
            if index + batch_size <= 253236:
                train_user_batch, train_item_batch, train_attr_batch,train_user_emb_batch= data_util.get_train_data(index, index + batch_size)
            index = index + batch_size
             
            _, G_loss_now = sess.run([G_solver, G_loss], feed_dict={attr:train_attr_batch})
        print( G_loss_now)
    if it % 1 == 0:
        test_item_batch,  test_attribute_vec= data_util.get_test_data()
        test_G_user = sess.run(gen_user_emb, feed_dict={attr:test_attribute_vec})
#        print( test_G_user[:10])
        p_at_10,p_at_20,M_at_10,M_at_20,G_at_10,G_at_20 = data_util.test(test_item_batch, test_G_user)
        if p_at_10 > max_p_at_10:
            max_p_at_10 = p_at_10
        p_10_to_save.append(p_at_10)
        if p_at_20 > max_p_at_20:
            max_p_at_20 = p_at_20
        p_20_to_save.append(p_at_20)
        if M_at_10 > max_M_at_10:
            max_M_at_10 = M_at_10
        M_10_to_save.append(M_at_10)
        if M_at_20 > max_M_at_20:
            max_M_at_20 = M_at_20
        M_20_to_save.append(M_at_20)
        if G_at_10 > max_G_at_10:
            max_G_at_10 = G_at_10
        G_10_to_save.append(G_at_10)
        if G_at_20 > max_G_at_20:
            max_G_at_20 = G_at_20
        G_20_to_save.append(G_at_20)
      
        print('movielens score: ', it, 'p_at_10 ', p_at_10, 'p_at_20', p_at_20,'mean_average_precision_at_10',M_at_10,'mean_average_precision_at_20',M_at_20,'nDCG_at_10',G_at_10,'nDCG_at_20',G_at_20)
        print('movielens max score: max p_at_10 ', max_p_at_10, 'p_at_20', max_p_at_20,'mean_average_precision_at_10',max_M_at_10,'mean_average_precision_at_20',max_M_at_20,'nDCG_at_10',max_G_at_10,'nDCG_at_20',max_G_at_20)
    if it % 100 == 0:
        pd.DataFrame(p_10_to_save).to_csv('generate_data/p10.csv')
        pd.DataFrame(p_20_to_save).to_csv('generate_data/p20.csv')
        pd.DataFrame(M_10_to_save).to_csv('generate_data/m10.csv')
        pd.DataFrame(M_20_to_save).to_csv('generate_data/m20.csv')
        pd.DataFrame(G_10_to_save).to_csv('generate_data/g10.csv')
        pd.DataFrame(G_20_to_save).to_csv('generate_data/g20.csv')
      

at start p_at_10 is 0.2342 p_at_20 0.2074 M_at_10 0.3791774527271226 M_at_20 0.34716994896267533 G_at_10 0.7436974789915967 G_at_20 0.8067226890756303
1.5204573
0.09703982
movielens score:  0 p_at_10  0.0605 p_at_20 0.0614 mean_average_precision_at_10 0.11541043004503389 mean_average_precision_at_20 0.11384156984749964 nDCG_at_10 0.296218487394958 nDCG_at_20 0.39915966386554624
movielens max score: max p_at_10  0.0605 p_at_20 0.0614 mean_average_precision_at_10 0.11541043004503389 mean_average_precision_at_20 0.11384156984749964 nDCG_at_10 0.296218487394958 nDCG_at_20 0.39915966386554624
1.5944895
0.29112667
movielens score:  1 p_at_10  0.2239 p_at_20 0.2037 mean_average_precision_at_10 0.3891505950925344 mean_average_precision_at_20 0.3556270594068721 nDCG_at_10 0.7310924369747899 nDCG_at_20 0.7899159663865546
movielens max score: max p_at_10  0.2239 p_at_20 0.2037 mean_average_precision_at_10 0.3891505950925344 mean_average_precision_at_20 0.3556270594068721 nDCG_at_10 0.731092436974

1.4275954
0.40417385
movielens score:  19 p_at_10  0.266 p_at_20 0.2338 mean_average_precision_at_10 0.4308066986715321 mean_average_precision_at_20 0.3922276285698131 nDCG_at_10 0.7184873949579832 nDCG_at_20 0.7521008403361344
movielens max score: max p_at_10  0.2668 p_at_20 0.2416 mean_average_precision_at_10 0.4400006371596258 mean_average_precision_at_20 0.3943908670034485 nDCG_at_10 0.773109243697479 nDCG_at_20 0.8361344537815126
1.401576
0.31672764
movielens score:  20 p_at_10  0.062 p_at_20 0.052 mean_average_precision_at_10 0.12446285723284022 mean_average_precision_at_20 0.11756000237939683 nDCG_at_10 0.29831932773109243 nDCG_at_20 0.36134453781512604
movielens max score: max p_at_10  0.2668 p_at_20 0.2416 mean_average_precision_at_10 0.4400006371596258 mean_average_precision_at_20 0.3943908670034485 nDCG_at_10 0.773109243697479 nDCG_at_20 0.8361344537815126
1.4099264
0.38940123
movielens score:  21 p_at_10  0.2355 p_at_20 0.1975 mean_average_precision_at_10 0.3993118113647574

1.387089
0.31479323
movielens score:  38 p_at_10  0.2676 p_at_20 0.2331 mean_average_precision_at_10 0.43669114008037074 mean_average_precision_at_20 0.4009850555906733 nDCG_at_10 0.7668067226890757 nDCG_at_20 0.819327731092437
movielens max score: max p_at_10  0.2685 p_at_20 0.2416 mean_average_precision_at_10 0.4400006371596258 mean_average_precision_at_20 0.4009850555906733 nDCG_at_10 0.7752100840336135 nDCG_at_20 0.8361344537815126
1.388951
0.33482948
movielens score:  39 p_at_10  0.2382 p_at_20 0.2063 mean_average_precision_at_10 0.38268916131267317 mean_average_precision_at_20 0.34890379907238733 nDCG_at_10 0.6827731092436975 nDCG_at_20 0.7415966386554622
movielens max score: max p_at_10  0.2685 p_at_20 0.2416 mean_average_precision_at_10 0.4400006371596258 mean_average_precision_at_20 0.4009850555906733 nDCG_at_10 0.7752100840336135 nDCG_at_20 0.8361344537815126
1.3942715
0.30549616
movielens score:  40 p_at_10  0.1059 p_at_20 0.1028 mean_average_precision_at_10 0.23225960655426

1.3907027
0.33920273
movielens score:  57 p_at_10  0.2676 p_at_20 0.2392 mean_average_precision_at_10 0.4399046404275996 mean_average_precision_at_20 0.3993084042588911 nDCG_at_10 0.7626050420168067 nDCG_at_20 0.8109243697478992
movielens max score: max p_at_10  0.2748 p_at_20 0.2416 mean_average_precision_at_10 0.4405825799235038 mean_average_precision_at_20 0.4009850555906733 nDCG_at_10 0.7899159663865546 nDCG_at_20 0.8361344537815126
1.3934971
0.41856045
movielens score:  58 p_at_10  0.1164 p_at_20 0.1006 mean_average_precision_at_10 0.23736001940458723 mean_average_precision_at_20 0.2103767794174095 nDCG_at_10 0.46638655462184875 nDCG_at_20 0.5672268907563025
movielens max score: max p_at_10  0.2748 p_at_20 0.2416 mean_average_precision_at_10 0.4405825799235038 mean_average_precision_at_20 0.4009850555906733 nDCG_at_10 0.7899159663865546 nDCG_at_20 0.8361344537815126
1.3961926
0.37514317
movielens score:  59 p_at_10  0.2651 p_at_20 0.235 mean_average_precision_at_10 0.4363704717998

1.388752
0.57950807
movielens score:  76 p_at_10  0.2674 p_at_20 0.2274 mean_average_precision_at_10 0.43542404097882537 mean_average_precision_at_20 0.400592682640941 nDCG_at_10 0.7836134453781513 nDCG_at_20 0.8172268907563025
movielens max score: max p_at_10  0.2748 p_at_20 0.2416 mean_average_precision_at_10 0.44226120243070777 mean_average_precision_at_20 0.40524419890847696 nDCG_at_10 0.7899159663865546 nDCG_at_20 0.8361344537815126
1.4088762
0.29576623
movielens score:  77 p_at_10  0.1586 p_at_20 0.1345 mean_average_precision_at_10 0.2686100478551209 mean_average_precision_at_20 0.24650712997018798 nDCG_at_10 0.542016806722689 nDCG_at_20 0.6113445378151261
movielens max score: max p_at_10  0.2748 p_at_20 0.2416 mean_average_precision_at_10 0.44226120243070777 mean_average_precision_at_20 0.40524419890847696 nDCG_at_10 0.7899159663865546 nDCG_at_20 0.8361344537815126
1.3931735
0.5702737
movielens score:  78 p_at_10  0.216 p_at_20 0.1701 mean_average_precision_at_10 0.3704356798936

...

1.4578929
0.19865645
movielens score:  722 p_at_10  0.2655 p_at_20 0.2375 mean_average_precision_at_10 0.4409981373501781 mean_average_precision_at_20 0.39918857575960676 nDCG_at_10 0.7773109243697479 nDCG_at_20 0.8046218487394958
movielens max score: max p_at_10  0.2748 p_at_20 0.2435 mean_average_precision_at_10 0.4450099643031815 mean_average_precision_at_20 0.4081714823903274 nDCG_at_10 0.7941176470588235 nDCG_at_20 0.8382352941176471
1.4077451
0.38291898
movielens score:  723 p_at_10  0.2655 p_at_20 0.2389 mean_average_precision_at_10 0.43990907077116564 mean_average_precision_at_20 0.3943884466040084 nDCG_at_10 0.7773109243697479 nDCG_at_20 0.8046218487394958
movielens max score: max p_at_10  0.2748 p_at_20 0.2435 mean_average_precision_at_10 0.4450099643031815 mean_average_precision_at_20 0.4081714823903274 nDCG_at_10 0.7941176470588235 nDCG_at_20 0.8382352941176471
1.4705191
0.20637268
movielens score:  724 p_at_10  0.266 p_at_20 0.241 mean_average_precision_at_10 0.43894045548

1.4054918
0.29800236
movielens score:  741 p_at_10  0.2655 p_at_20 0.2375 mean_average_precision_at_10 0.441574284475695 mean_average_precision_at_20 0.3993543088145896 nDCG_at_10 0.7773109243697479 nDCG_at_20 0.8046218487394958
movielens max score: max p_at_10  0.2748 p_at_20 0.2435 mean_average_precision_at_10 0.4450099643031815 mean_average_precision_at_20 0.4081714823903274 nDCG_at_10 0.7941176470588235 nDCG_at_20 0.8382352941176471
1.4343958
0.26558992
movielens score:  742 p_at_10  0.266 p_at_20 0.2389 mean_average_precision_at_10 0.4394073088891641 mean_average_precision_at_20 0.39440322114686266 nDCG_at_10 0.7773109243697479 nDCG_at_20 0.8046218487394958
movielens max score: max p_at_10  0.2748 p_at_20 0.2435 mean_average_precision_at_10 0.4450099643031815 mean_average_precision_at_20 0.4081714823903274 nDCG_at_10 0.7941176470588235 nDCG_at_20 0.8382352941176471
1.4233544
0.2669859
movielens score:  743 p_at_10  0.266 p_at_20 0.2389 mean_average_precision_at_10 0.43862092952260

1.4244354
0.29253817
movielens score:  760 p_at_10  0.2674 p_at_20 0.2389 mean_average_precision_at_10 0.4403455476105786 mean_average_precision_at_20 0.3963203579461038 nDCG_at_10 0.7668067226890757 nDCG_at_20 0.8046218487394958
movielens max score: max p_at_10  0.2748 p_at_20 0.2435 mean_average_precision_at_10 0.4450099643031815 mean_average_precision_at_20 0.4081714823903274 nDCG_at_10 0.7941176470588235 nDCG_at_20 0.8382352941176471
1.4302111
0.27125412
movielens score:  761 p_at_10  0.2674 p_at_20 0.2389 mean_average_precision_at_10 0.4403455476105786 mean_average_precision_at_20 0.39595183770584347 nDCG_at_10 0.7668067226890757 nDCG_at_20 0.8046218487394958
movielens max score: max p_at_10  0.2748 p_at_20 0.2435 mean_average_precision_at_10 0.4450099643031815 mean_average_precision_at_20 0.4081714823903274 nDCG_at_10 0.7941176470588235 nDCG_at_20 0.8382352941176471
1.4188643
0.2693371
movielens score:  762 p_at_10  0.266 p_at_20 0.2407 mean_average_precision_at_10 0.440540381165

1.4367597
0.34983572
movielens score:  779 p_at_10  0.2693 p_at_20 0.2368 mean_average_precision_at_10 0.4419235144454607 mean_average_precision_at_20 0.402736185838255 nDCG_at_10 0.7794117647058824 nDCG_at_20 0.8067226890756303
movielens max score: max p_at_10  0.2748 p_at_20 0.2435 mean_average_precision_at_10 0.4450099643031815 mean_average_precision_at_20 0.4081714823903274 nDCG_at_10 0.7941176470588235 nDCG_at_20 0.8382352941176471
1.4551097
0.24432355
movielens score:  780 p_at_10  0.266 p_at_20 0.2375 mean_average_precision_at_10 0.4377673738146052 mean_average_precision_at_20 0.3965751592677427 nDCG_at_10 0.7773109243697479 nDCG_at_20 0.8046218487394958
movielens max score: max p_at_10  0.2748 p_at_20 0.2435 mean_average_precision_at_10 0.4450099643031815 mean_average_precision_at_20 0.4081714823903274 nDCG_at_10 0.7941176470588235 nDCG_at_20 0.8382352941176471
1.4210088
0.36100864
movielens score:  781 p_at_10  0.2697 p_at_20 0.2396 mean_average_precision_at_10 0.4367708396189

1.4714725
0.18400265
movielens score:  798 p_at_10  0.2672 p_at_20 0.2396 mean_average_precision_at_10 0.4436827664266765 mean_average_precision_at_20 0.4025297959252054 nDCG_at_10 0.7668067226890757 nDCG_at_20 0.8130252100840336
movielens max score: max p_at_10  0.2748 p_at_20 0.2435 mean_average_precision_at_10 0.4450099643031815 mean_average_precision_at_20 0.4081714823903274 nDCG_at_10 0.7941176470588235 nDCG_at_20 0.8382352941176471
1.4124262
0.4044777
movielens score:  799 p_at_10  0.2672 p_at_20 0.2375 mean_average_precision_at_10 0.43810686278479644 mean_average_precision_at_20 0.4002571653274045 nDCG_at_10 0.7836134453781513 nDCG_at_20 0.8046218487394958
movielens max score: max p_at_10  0.2748 p_at_20 0.2435 mean_average_precision_at_10 0.4450099643031815 mean_average_precision_at_20 0.4081714823903274 nDCG_at_10 0.7941176470588235 nDCG_at_20 0.8382352941176471
